In [2]:
import pypyodbc as odbc #!pip install pypyodbc
import pandas as pd
import time
import os

print(odbc.drivers())

['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client RDA 11.0', 'SQL Server Native Client RDA 11.0']


#### Needed Inputs:

In [3]:
driverName = "SQL Server" 
serverName = "DESKTOP-UVMKETP\SQLEXPRESS"
dataBaseName = "SQL_PowerBI_Test"
username = "jvenncpe"
pwd = "#Juv1994*"
tableName = "olist_order_reviews_dataset"

csv_FolderName = r"olist_order_reviews_dataset"
csv_FilePaTH = r"G:\00 Work Files\02 Work Trainings & Apps\00 Project Portfolio\Ecommerce_Amazon (Python, SQL, PowerBI)\CSV Files\Amazon_Olist"
csv_FileName = r"olist_order_reviews_dataset.csv" #To be used for column name reference in SQL Table

### Custom Functions

In [4]:
def updateSQL():
    try:
        conn.commit()
        return print("SQL Updated Successfully!")
        
    except Exception as e:
        return print(f"Error occurred: {e}")

def SQL_createTable_columnName(column):
    column_names = []
    for x in column:
        x = f'[{x}] VARCHAR(MAX)'
        column_names.append(x)
    return ', '.join(column_names)  # Join the column names with commas

def SQL_createTable_query(tableName,tableColumns):
    createTable_query = f"\
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{tableName}')\
    BEGIN\
        CREATE TABLE {dataBaseName}.dbo.{tableName}({tableColumns})\
    END".strip()
    return createTable_query

def bulk_insert(datapath_file, SQL_Table):
    insertBulk_query = f"""
        BULK INSERT {SQL_Table}
        FROM '{datapath_file}'
        WITH
        (
            FIRSTROW = 2,
            FIELDTERMINATOR = ',', 
            ROWTERMINATOR = '\n',  
            TABLOCK
        ) 
    """.strip()
    return insertBulk_query

#### Adhoc SQL Configuration

In [5]:
connection_string = f"""
    DRIVER={{{driverName}}};
    SERVER={serverName};
    DATABASE={dataBaseName};
    Trust_Connection=yes;
    UID={username};
    PWD={pwd};
"""
try:
    conn = odbc.connect(connection_string)
    cursor = conn.cursor()
    print("Connection successful!")

except Exception as e:
    print(f"Error: {e}")


Connection successful!


#### Main Function

In [6]:
if __name__ == '__main__':
    csv_filePaTH_REF = os.path.join(csv_FilePaTH,csv_FolderName,csv_FileName)   
    csv_FoldderPath = os.path.join(csv_FilePaTH, csv_FolderName)
    
    df_productDetails = pd.read_csv(csv_filePaTH_REF, encoding='latin1', na_values=[""])
    
    for columnName in df_productDetails.columns:
        columnName_After = columnName.replace(".","")
        columnName_After = columnName_After.replace(" ","_")
        df_productDetails.rename(columns={columnName: columnName_After}, inplace=True)
    
    cursor = conn.cursor()
    tableColumns = SQL_createTable_columnName(df_productDetails.columns)
    
    try:
        with cursor:
            cursor.execute(SQL_createTable_query(tableName,tableColumns))
            updateSQL()
    except Exception as e:
        print(e)
        conn.rollback()
        print('Transaction rollback')
    
    data_files = os.listdir(csv_FoldderPath)
    SQL_Table = f'{dataBaseName}.dbo.{tableName}'
    
    cursor = conn.cursor()
    try:
        with cursor:
            print(f"Target SQL Table: {SQL_Table}")
            print(f"---------------------------")
            print(f"Files to Upload:")
            
            for data_file in data_files:
                if data_file.endswith('.csv'):
                    cursor.execute(bulk_insert(os.path.join(csv_FoldderPath, data_file), SQL_Table))
                    print(data_file)
                    
            print(f"---------------------------")
            updateSQL()
            
    except Exception as e:
        print(e)
        conn.rollback()
        print('Transaction rollback')

SQL Updated Successfully!
Target SQL Table: SQL_PowerBI_Test.dbo.olist_order_reviews_dataset
---------------------------
Files to Upload:
olist_order_reviews_dataset.csv
---------------------------
SQL Updated Successfully!


In [7]:
df_productDetails.head(20)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,18/01/2018 0:00,18/01/2018 21:46
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,10/03/2018 0:00,11/03/2018 3:05
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,17/02/2018 0:00,18/02/2018 14:36
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,21/04/2017 0:00,21/04/2017 22:02
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,ParabÃ©ns lojas lannister adorei comprar pela ...,01/03/2018 0:00,02/03/2018 10:26
5,15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,NaN,NaN,13/04/2018 0:00,16/04/2018 0:39
6,07f9bee5d1b850860defd761afa7ff16,e48aa0d2dcec3a2e87348811bcfdf22b,5,NaN,NaN,16/07/2017 0:00,18/07/2017 19:30
7,7c6400515c67679fbee952a7525281ef,c31a859e34e3adac22f376954e19b39d,5,NaN,NaN,14/08/2018 0:00,14/08/2018 21:36
8,a3f6f7f6f433de0aefbb97da197c554c,9c214ac970e84273583ab523dfafd09b,5,NaN,NaN,17/05/2017 0:00,18/05/2017 12:05
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,22/05/2018 0:00,23/05/2018 16:45
